In [1]:
spark.version

'3.1.3'

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
directory = 'gs://msca-bdp-tweets/final_project'
file = '/*.json'
path = directory + file
# path = directory

In [5]:
%%time
tweets_df = spark.read.json(path)

CPU times: user 1.23 s, sys: 290 ms, total: 1.52 s
Wall time: 6min 57s


22/12/07 02:41:21 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
tweets_df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries
null,Mon Sep 26 19:06:...,null,"{[], null, [], []...",null,null,0,false,low,null,1574475610692337664,1574475610692337664,null,null,null,null,null,true,en,null,null,0,"{null, Sat Sep 24...",1573804912265281537,1573804912265281537,{twitter.com/notb...,why does the univ...,0,0,RT,JoshuaPotash,"{null, Mon Sep 26...","<a href=""http://t...",RT @JoshuaPotash:...,1664219212588,false,People say free c...,"{false, Mon Dec 0...",null,null
null,Mon Sep 26 19:06:...,null,"{[], null, [], []...",null,null,0,false,low,null,1574475611724124160,1574475611724124160,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,DeAngelisCorey,"{null, Mon Sep 26...","<a href=""http://t...",RT @DeAngelisCore...,1664219212834,false,@scottbraddock @T...,"{false, Wed Oct 0...",null,null
null,Mon Sep 26 19:06:...,null,"{[], null, [], []...",null,null,0,false,low,null,1574475611900461068,1574475611900461068,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,TheKingOlu,"{null, Mon Sep 26...","<a href=""http://t...",RT @TheKingOlu: G...,1664219212876,false,Go back to school...,"{false, Tue Feb 0...",null,null
null,Mon Sep 26 19:06:...,null,"{[], null, [], []...",null,null,0,false,low,null,1574475612407971848,1574475612407971848,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,Rockstarscars,"{null, Mon Sep 26...","<a href=""http://t...",RT @Rockstarscars...,1664219212997,false,Happy 77th Birthd...,"{false, Mon Mar 1...",null,null
null,Mon Sep 26 19:06:...,"[10, 140]","{[], null, [], [{...",null,"{[10, 279], {[], ...",0,false,low,null,1574475612361670657,1574475612361670657,mercnews,1574473995138875400,1574473995138875400,10433782,10433782,false,en,null,null,0,null,null,null,null,null,0,0,,LivermorePolice t...,null,"<a href=""http://t...",@mercnews To conc...,1664219212986,true,@mercnews To conc...,"{false, Sat Jun 0...",null,null


In [6]:
import pandas as pd
pd.set_option("max_colwidth", 1001)

In [24]:
tweets_df.select('user').limit(1).toPandas()

,user
0,"(False, Thu Mar 05 21:39:02 +0000 2009, False, False, 💙⚾️💛🏀💜🏒🖤, 14148, 71, 637, True, 22983536, 22983536, False, 0, Los Angeles, shiaoma, 000000, http://abs.twimg.com/images/themes/theme10/bg.gif, https://abs.twimg.com/images/themes/theme10/bg.gif, False, None, http://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, https://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, ABB8C2, 000000, 000000, 000000, False, False, shiaoma, 2248, none, None, False, [])"


In [7]:
data = tweets_df.select([tweets_df.created_at,
                         tweets_df.place.country.alias("country"),
                         tweets_df.id,
                         tweets_df.user, 
                         tweets_df.user['id'].alias("user_id"),
                         tweets_df.user['name'].alias("user_name"), 
                         tweets_df.user['description'].alias("user_description"), 
                         tweets_df.user['followers_count'].alias("followers_count"), 
                         tweets_df.retweeted_status.quote_count.alias("quote_count"),
                         tweets_df.retweeted_status,
                         tweets_df.retweeted_status.retweet_count.alias("retweet_count"),
                         tweets_df.retweeted_status.favorite_count.alias("favorite_count"),
                         tweets_df.text, 
                         tweets_df.user.verified.alias("verified_user")])

In [8]:
import pyspark.sql.functions as f

In [9]:
data = data.withColumn("text",f.lower(f.col("text")))

In [11]:
type(data)

22/11/24 23:13:47 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 11 for reason Container marked as failed: container_1669313351537_0002_01_000011 on host: hub-msca-bdp-dphub-students-paraut-sw-840c.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/11/24 23:13:47 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 11 on hub-msca-bdp-dphub-students-paraut-sw-840c.c.msca-bdp-students.internal: Container marked as failed: container_1669313351537_0002_01_000011 on host: hub-msca-bdp-dphub-students-paraut-sw-840c.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.


In [26]:
data.limit(1).toPandas()

,created_at,country,id,user,user_id,user_name,user_description,followers_count,quote_count,retweeted_status,retweet_count,favorite_count,text,verified_user
0,Tue May 24 22:09:56 +0000 2022,None,1529223198327287808,"(False, Thu Mar 05 21:39:02 +0000 2009, False, False, 💙⚾️💛🏀💜🏒🖤, 14148, 71, 637, True, 22983536, 22983536, False, 0, Los Angeles, shiaoma, 000000, http://abs.twimg.com/images/themes/theme10/bg.gif, https://abs.twimg.com/images/themes/theme10/bg.gif, False, None, http://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, https://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, ABB8C2, 000000, 000000, 000000, False, False, shiaoma, 2248, none, None, False, [])",22983536,shiaoma,💙⚾️💛🏀💜🏒🖤,71,355,"(None, Tue May 24 21:54:24 +0000 2022, [0, 140], ([], None, [], [Row(display_url='twitter.com/i/web/status/1…', expanded_url='https://twitter.com/i/web/status/1529219289005400064', indices=[117, 140], url='https://t.co/7XLz71jM8u')], []), None, ([0, 279], ([], [Row(additional_media_info=Row(description=None, embeddable=None, monetizable=False, title=None), description=None, display_url='pic.twitter.com/9fkJ13vWGd', expanded_url='https://twitter.com/ABC/status/1529219289005400064/video/1', id=1529219222970281984, id_str='1529219222970281984', indices=[280, 303], media_url='http://pbs.twimg.com/amplify_video_thumb/1529219222970281984/img/rr2UoSQsvksDKCP-.jpg', media_url_https='https://pbs.twimg.com/amplify_video_thumb/1529219222970281984/img/rr2UoSQsvksDKCP-.jpg', sizes=Row(large=Row(h=540, resize='fit', w=540), medium=Row(h=540, resize='fit', w=540), small=Row(h=540, resize='fit', w=540), thumb=Row(h=150, resize='crop', w=150)), source_status_id=None, source_status_id_str=None, sourc...",3008,8422,"rt @abc: “why are you here?!""\n\na furious sen. chris murphy demands answers from senators following texas school shooting.\n\n“why do you spen…",False


In [10]:
education_data = data.filter((data.text.contains("education"))\
                          | (data.text.contains("university"))\
                          | (data.text.contains("school"))\
                          | (data.text.contains("degree"))\
                          | (data.text.contains("book"))\
                          | (data.text.contains("public school"))\
                          | (data.text.contains("higher education"))\
                          | (data.text.contains("science"))\
                          | (data.text.contains("math"))
                          | (data.text.contains("online learning"))\
                          | (data.text.contains("Secondary education"))\
                          | (data.text.contains("k12"))\
                          | (data.text.contains("curriculum"))\
                          | (data.text.contains("stem"))\
                          | (data.text.contains("textbook"))\
                          | (data.text.contains("science of learning"))\
                          | (data.text.contains("online learning"))\
                          | (data.text.contains("ai-driven learning"))\
                          | (data.text.contains("education trends"))\
                          | (data.text.contains("education platforms"))\
                          | (data.text.contains("homeschooling"))\
                          | (data.text.contains("hybrid-learning"))\
                          | (data.text.contains("k-12 cybersecurity"))\
                          | (data.text.contains("asynchronous learning"))\
                          | (data.text.contains("augmented & virtual reality in the classroom"))\
                          | (data.text.contains("social-emotional learning"))\
                          | (data.text.contains("game-based learning")))


#https://www.mchdata.com/blog/7-k-12-education-trends-for-2022

In [13]:
education_data.limit(10).toPandas()

,created_at,country,id,user,user_id,user_name,user_description,followers_count,quote_count,retweeted_status,retweet_count,favorite_count,text,verified_user
0,Tue May 24 22:09:56 +0000 2022,None,1529223198327287808,"(False, Thu Mar 05 21:39:02 +0000 2009, False, False, 💙⚾️💛🏀💜🏒🖤, 14148, 71, 637, True, 22983536, 22983536, False, 0, Los Angeles, shiaoma, 000000, http://abs.twimg.com/images/themes/theme10/bg.gif, https://abs.twimg.com/images/themes/theme10/bg.gif, False, None, http://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, https://pbs.twimg.com/profile_images/2301998319/image_normal.jpg, ABB8C2, 000000, 000000, 000000, False, False, shiaoma, 2248, none, None, False, [])",22983536,shiaoma,💙⚾️💛🏀💜🏒🖤,71,355.0,"(None, Tue May 24 21:54:24 +0000 2022, [0, 140], ([], None, [], [Row(display_url='twitter.com/i/web/status/1…', expanded_url='https://twitter.com/i/web/status/1529219289005400064', indices=[117, 140], url='https://t.co/7XLz71jM8u')], []), None, ([0, 279], ([], [Row(additional_media_info=Row(description=None, embeddable=None, monetizable=False, title=None), description=None, display_url='pic.twitter.com/9fkJ13vWGd', expanded_url='https://twitter.com/ABC/status/1529219289005400064/video/1', id=1529219222970281984, id_str='1529219222970281984', indices=[280, 303], media_url='http://pbs.twimg.com/amplify_video_thumb/1529219222970281984/img/rr2UoSQsvksDKCP-.jpg', media_url_https='https://pbs.twimg.com/amplify_video_thumb/1529219222970281984/img/rr2UoSQsvksDKCP-.jpg', sizes=Row(large=Row(h=540, resize='fit', w=540), medium=Row(h=540, resize='fit', w=540), small=Row(h=540, resize='fit', w=540), thumb=Row(h=150, resize='crop', w=150)), source_status_id=None, source_status_id_str=None, sourc...",3008.0,8422.0,"rt @abc: “why are you here?!""\n\na furious sen. chris murphy demands answers from senators following texas school shooting.\n\n“why do you spen…",False
1,Tue May 24 22:09:56 +0000 2022,None,1529223197253435392,"(False, Thu May 19 00:00:53 +0000 2022, True, False, None, 0, 0, 31, False, 1527076684557451264, 1527076684557451264, False, 0, None, High School Sports, F5F8FA, , , False, https://pbs.twimg.com/profile_banners/1527076684557451264/1653379910, http://pbs.twimg.com/profile_images/1529010953886453760/u8Pj5QuF_normal.jpg, https://pbs.twimg.com/profile_images/1529010953886453760/u8Pj5QuF_normal.jpg, 1DA1F2, C0DEED, DDEEF6, 333333, True, False, Gabriel50407921, 160, none, None, False, [])",1527076684557451264,High School Sports,None,0,NaN,None,NaN,NaN,indiana high school softball | live streaming\ntba vs fishers\neastbrook vs oak hill\ntba vs wabash\nsouth newton vs cl… https://t.co/qvvqquuog8,False
2,Tue May 24 22:09:56 +0000 2022,None,1529223198478311425,"(False, Fri Jun 24 02:59:29 +0000 2011, False, False, Mother/daughter/sister/survivor \nMe, Myself, & I are a Power Couple., 13661, 27, 192, False, 323002943, 323002943, False, 0, Texas, FullyDedicated2Thee, B2DFDA, http://abs.twimg.com/images/themes/theme13/bg.gif, https://abs.twimg.com/images/themes/theme13/bg.gif, False, None, http://pbs.twimg.com/profile_images/816862238468440064/aGt8AKKr_normal.jpg, https://pbs.twimg.com/profile_images/816862238468440064/aGt8AKKr_normal.jpg, 93A644, EEEEEE, FFFFFF, 333333, True, False, 2Short2Sweet, 1833, none, None, False, [])",323002943,FullyDedicated2Thee,"Mother/daughter/sister/survivor \nMe, Myself, & I are a Power Couple.",27,1857.0,"(None, Tue May 24 20:42:47 +0000 2022, None, ([], None, [], [Row(display_url='twitter.com/i/web/status/1…', expanded_url='https://twitter.com/i/web/status/1529201265879703553', indices=[116, 139], url='https://t.co/Fbyc0Dt4pE')], []), None, ([0, 174], ([], None, [], [], []), None, Fifteen have been killed in a shooting at Robb elementary school, according to Governor Greg Abbott — 14 students and 1 teacher. \n \nAbbott said the shooter is also deceased.), 8185, False, low, None, 1529201265879703553, 1529201265879703553, None, None, None, None, None, False, en, None, None, 1857, None, No

In [11]:
education_df = education_data.na.fill(value=0,subset=["retweet_count", "quote_count", "favorite_count"])

In [12]:
classification_df = education_df.na.drop(subset = ['user_description'])

In [12]:
classification_df.count()

52841052

In [15]:
influence = classification_df.select([classification_df.user_id, 
                                       classification_df.retweeted_status,
                                       classification_df.retweet_count,
                                       classification_df.quote_count,
                                       classification_df.favorite_count])

In [16]:
influence = influence\
    .withColumn("retweet_quote_count", col("retweet_count")+col("quote_count"))\
    .withColumn("total_engagement", col("retweet_count")+col("quote_count")+col("favorite_count"))

In [17]:
total_count = influence.groupBy('user_id').count().alias('total_count')

In [18]:
joined_df = influence.join(total_count, "user_id" ,"left")

In [19]:
joined_df2 = joined_df.withColumn("influence", (col("retweet_quote_count")/col("total_engagement")) * col("count"))

In [20]:
joined_df3 = joined_df2.na.fill(value=0,subset=["influence"])

In [21]:
joined_df4 = joined_df3.withColumn("influence", when(joined_df3.retweeted_status.isNull(), joined_df3.influence*3).otherwise(joined_df3.influence))

In [22]:
influencers = joined_df4.select(['user_id','influence']).groupBy("user_id").agg(avg("influence").alias("avg_influence"))

In [23]:
joined_df5 = influencers.join(joined_df4, 
                       "user_id",
                       "left")

In [24]:
joined_df6 = joined_df5.drop("retweet_count")\
    .drop("quote_count")\
    .drop("favorite_count")\
    .drop("influence")\
    .drop("retweeted_status")\
    .drop("retweet_quote_count")\
    .drop("total_engagement")

In [25]:
joined_df7 = joined_df6.dropDuplicates(['user_id'])

In [26]:
education_influence = classification_df.join(joined_df7, 
                       "user_id",
                       "left")

In [27]:
from pyspark.sql import functions as f

In [28]:
education_influence2 = education_influence.withColumn("Organization",
                        f.when(education_influence.verified_user == 'false', 'Other') \
                        .when((education_influence.user_description.contains('university')) | (education_influence.user_name.contains('university')) | (education_influence.user_name.contains('University')) | (education_influence.user_name.contains('School of')), 'University')
                        .when((education_influence.user_description.contains('school')) | (education_influence.user_name.contains('school')) | (education_influence.user_name.contains('School')) , 'School')  
                        .when((education_influence.user_description.contains('nonprofit organization')), 'nonprofit organization')                             
                        .when((education_influence.user_description.contains('gov')) | (education_influence.user_name.contains('gov')) | (education_influence.user_name.contains('Department of')) \
                              | (education_influence.user_description.contains('president')) | (education_influence.user_name.contains('president')) \
                              | (education_influence.user_description.contains('senat')) | (education_influence.user_name.contains('senat')) \
                              | (education_influence.user_description.contains('congress')) | (education_influence.user_name.contains('congress')) \
                              | (education_influence.user_description.contains('prime minister')) | (education_influence.user_name.contains('prime minister')) \
                              | (education_influence.user_description.contains('parliament')) | (education_influence.user_name.contains('parliament')) \
                              | (education_influence.user_description.contains('vp')) | (education_influence.user_name.contains('vp')) \
                              | (education_influence.user_description.contains('white house')) | (education_influence.user_name.contains('white house')) \
                              | (education_influence.user_description.contains('mayor')) | (education_influence.user_name.contains('mayor')) \
                              | (education_influence.user_description.contains('democr')) | (education_influence.user_name.contains('democr')) \
                              | (education_influence.user_description.contains('republic')) | (education_influence.user_name.contains('republic')), 'Government') \
                        .when((education_influence.user_description.contains('news')) | (education_influence.user_name.contains('news')) \
                              | (education_influence.user_description.contains('News')) | (education_influence.user_name.contains('News')) \
                              | (education_influence.user_description.contains('daily')) | (education_influence.user_name.contains('daily')) \
                              | (education_influence.user_description.contains('times')) | (education_influence.user_name.contains('times')) \
                              | (education_influence.user_description.contains('journal')) | (education_influence.user_name.contains('journal')), 'News') \
                        .when(((education_influence.user_description.contains('actor')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('singer')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('writer')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('play')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('musician')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('actress')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('golfer')) & (education_influence.followers_count > 30000)) \
                              | ((education_influence.user_description.contains('author')) & (education_influence.followers_count > 30000)), 'Celebrity') \
                        .when(education_influence.followers_count >= 10000, 'Influencer')) 

In [29]:
education_influence2 = education_influence2.na.fill(value='Other',subset=["Organization"])

In [30]:
education_influence3 = education_influence2.dropDuplicates(['user_id'])

In [31]:
from google.cloud import storage

In [32]:
bucket_read = 'msca-bdp-data-open'

In [33]:
bucket_write = 'msca-bdp-students-bucket'

In [34]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [ ]:
education_influence3.write.format("parquet").\
mode('overwrite').\
save('gs://' + bucket_write + '/shared_data/paraut/tweets')

print('done')

done
